In [1]:
import pandas as pd
import numpy as np
import datetime
import psycopg2
import random
import matplotlib.pyplot as plt
import difflib
from geopy import distance
from psycopg2 import Error
from psycopg2 import sql
import time

In [13]:
# Подключение к БД
conn = psycopg2.connect(user = "",
                      password = "",
                      host = "",
                      port = "",
                      database = "")

In [12]:
with conn.cursor() as cursor:
    conn.autocommit = True
    select = sql.SQL('''
    with metro as (select po.obj_id, CASE WHEN MIN(t.metro_station_transport_length) <
     MIN(t.metro_station_walking_length) THEN MIN(t.metro_station_transport_length)
    ELSE MIN(t.metro_station_walking_length) END metro_length
    from emart.portal_obj po
    left join emart.portal_obj_transport_dist t on t.obj_id = po.obj_id
    group by po.obj_id),
    
    prices as (select aocc.obj_id, AVG(aocc.price_conclude_amt/aocc.area_conclude_sq) price
    from emarti.act_obj_concluded_contract aocc
    where aocc.contract_elem_type_cd = 1 and aocc.price_conclude_amt != 0 and aocc.area_conclude_sq != 0
    group by obj_id)

    select po.obj_id, ao.obj_title_nm, ao.rpd_region_cd, ao.obj_lk_latitude, ao.obj_lk_longitude, ao.isliving_flg, 
    ao.obj_ready_cd, ao.isbuild_flg, ao.obj_comiss_dt, ao.obj_lk_class_cd, ao.obj_floor_max,
    ao.obj_addr_area_type_cd, metro.metro_length, prices.price

    from emart.portal_obj po
    left join metro on metro.obj_id = po.obj_id
    left join emarti.act_obj ao on ao.obj_id = po.obj_id
    left join prices on prices.obj_id = po.obj_id
    WHERE ao.pb_flg = 0''')
    cursor.execute(select)
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    data_raw = pd.DataFrame(list(data), columns=columns)
    data_raw = data_raw.set_index('obj_id')

KeyboardInterrupt: 

In [41]:
print(len(data_raw))

29187


In [18]:
start_time = time.time()
# Группируем по региону
region_set = set(data_raw['rpd_region_cd'])
print("--- %s seconds ---" % (time.time() - start_time))
data_start = {}
for reg in region_set:
    data_start[reg] = pd.DataFrame(columns=data_raw.columns)
for i in data_raw.index.tolist():
    data_start[data_raw.loc[i]['rpd_region_cd']] = data_start[data_raw.loc[i]['rpd_region_cd']].append(data_raw.loc[i])
data = data_start.copy()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.05045771598815918 seconds ---
--- 309.0123791694641 seconds ---


In [31]:
def similarity(s1, s2):
    matcher = difflib.SequenceMatcher(None, s1.lower(), s2.lower())
    return matcher.ratio()

def dist_calc (row1,row2):
    return distance.great_circle((row1[0],row1[1]), (row2[0],row2[1])).km

def data_prepare(data,region,home,N):
    # Заменяем NaN на нули
    data[region]['obj_lk_class_cd'].fillna(0, inplace=True)
    data[region]['obj_floor_max'].fillna(0, inplace=True)
#     data[region]['obj_lk_wall_material_cd'].fillna(0, inplace=True)
    data[region]['metro_length'].fillna(0, inplace=True)
    data[region]['price'].fillna(0, inplace=True)
    data[region]['obj_lk_latitude'].fillna(0, inplace=True)
    data[region]['obj_lk_longitude'].fillna(0, inplace=True)
    data[region]['obj_title_nm'].fillna('None', inplace=True)
    # Приводим дату ввода в количество дней до ввода
    data[region]['obj_comiss_dt'] = pd.to_datetime(data[region]['obj_comiss_dt']).dt.date
    data[region]['years_to_ready'] = (((data[region]['obj_comiss_dt'] - datetime.datetime.now().date()).dt.days) / 365).astype(float)
    # Рассчитываем расстояние до дома
#     new_df = pd.concat([data[region]['obj_lk_latitude'], data[region]['obj_lk_longitude']], axis=1)
#     data[region]['distance'] = new_df.apply(lambda row: dist_calc (row,(data[region].loc[home]['obj_lk_latitude'],
#                       data[region].loc[home]['obj_lk_longitude'])), axis=1)
    # Приводим типы данных
    data[region]['obj_lk_latitude'] = data[region]['obj_lk_latitude'].astype(float)
    data[region]['obj_lk_longitude'] = data[region]['obj_lk_longitude'].astype(float)
    data[region]['isliving_flg'] = data[region]['isliving_flg'].astype(int)
    data[region]['obj_ready_cd'] = data[region]['obj_ready_cd'].astype(int)
    data[region]['isbuild_flg'] = data[region]['isbuild_flg'].astype(int)
    data[region]['obj_lk_class_cd'] = data[region]['obj_lk_class_cd'].astype(int)
    data[region]['obj_floor_max'] = data[region]['obj_floor_max'].astype(int)
#     data[region]['obj_lk_wall_material_cd'] = data[region]['obj_lk_wall_material_cd'].astype(int)
    data[region]['obj_addr_area_type_cd'] = data[region]['obj_addr_area_type_cd'].astype(int)
    data[region]['metro_length'] = data[region]['metro_length'].astype(float)
    data[region]['price'] = (data[region]['price'] / 100000).astype(float)
    # Удаляем ненужные столбцы
    del data[region]['obj_comiss_dt']
    del data[region]['rpd_region_cd']
    
    # Фильтруем
    isliving = data[region]['isliving_flg'].loc[home]
    isbuild = data[region]['isbuild_flg'].loc[home]
    obj_lk_class = data[region]['obj_lk_class_cd'].loc[home]
    floor = data[region]['obj_floor_max'].loc[home]
    
    data_isliving = data[region][data[region]['isliving_flg'] == isliving]
    if len(data_isliving) > N+1:
        data[region] = data_isliving
    data_isbuild = data[region][data[region]['isbuild_flg'] == isbuild]
    if len(data_isbuild) > N+1:
        data[region] = data_isbuild
    if obj_lk_class == 4:
        data_class4 = data[region][data[region]['obj_lk_class_cd'] == 4]
        if len(data_class4) > N+1:
            data[region] = data_class4
    else:
        data_classnot4 = data[region][data[region]['obj_lk_class_cd'] != 4]
        if len(data_classnot4) > N+1:
            data[region] = data_classnot4       
    if floor <= 6:
        data_floor16 = data[region][data[region]['obj_floor_max'] <= 6]
        if len(data_floor16) > N+1:
            data[region] = data_floor16
    else:
        data_floor6plus = data[region][data[region]['obj_floor_max'] > 6]
        if len(data_floor6plus) > N+1:
            data[region] = data_floor6plus
    return data

def corr_recs(data,region,home,seen,N):
    # Рассчет корреляций
    obj_names = data[region]['obj_title_nm']
    obj_latitude = data[region]['obj_lk_latitude']
    obj_longitude = data[region]['obj_lk_longitude']
    del data[region]['obj_title_nm']
    del data[region]['obj_lk_latitude']
    del data[region]['obj_lk_longitude']
    corrs = data[region].T.corr()[home].sort_values(ascending=False).rename('corrs')
    data_res = data[region].query('index in @corrs.index')
    data_res = data_res.join(corrs)
    # Фильтруем по названиям домов
    recs = [home]
    sim_recs = []
    for res in data_res.sort_values('corrs', ascending=False).index:
        flg = 0
        for rec in recs:
            if similarity(obj_names.loc[res],obj_names.loc[rec]) > 0.6:
                flg = 1
                sim_recs.append(res)
        if flg == 0:
            if dist_calc((obj_latitude.loc[home], obj_longitude.loc[home]),
                    (obj_latitude.loc[res], obj_longitude.loc[res])) < 4:
                recs.append(res)
            elif len(recs) < N+1:
                recs.append(res)
            else:
                continue
        if (len(recs) == N+3):
            break
    recs.remove(home)    
    if len(recs) < N:
        recs = recs + sim_recs[:N-len(recs)]
    try:
        return [recs[i] for i in random.sample(range(len(recs)), N)]
    except:
        return recs    

In [42]:
start_time = time.time()
values = []
seen = []
N = 5 # количество рекомендаций
for region in region_set:
    for home in data_start[region].index:
        data[region] = data_start[region].copy(deep=True)
        data = data_prepare(data,region,home,N)
        values.append((home, list(corr_recs(data,region,home,seen,N))))
print("--- %s seconds ---" % (time.time() - start_time))

--- 3901.48379278183 seconds ---


In [43]:
for row in values:
    print(row)

(13030, [29596, 33530, 639, 31271, 31387])
(13025, [639, 29596, 31387, 31271, 33530])
(42742, [44144, 41821, 38067, 12618, 38635])
(13565, [13563, 13566, 29595, 13565, 31394])
(13028, [31387, 639, 29596, 31271, 33530])
(13576, [13577, 13578, 13574, 29595, 31395])
(13548, [31272, 639, 29596, 33530, 31388])
(31264, [14937, 12615, 13276, 574, 32158])
(42744, [42744, 38550, 41057, 38375, 37115])
(35106, [33532, 574, 26385, 14177, 12615])
(30565, [31270, 24726, 14178, 14937, 13276])
(38880, [41057, 37113, 38376, 36629, 38880])
(13549, [639, 38549, 31271, 31390, 29596])
(31389, [38549, 13552, 639, 29596, 31272])
(9879, [33530, 29596, 31387, 31272, 639])
(13029, [33530, 639, 31271, 31387, 29596])
(12615, [31265, 13276, 14177, 574, 33729])
(22860, [33530, 31387, 29596, 31272, 639])
(13556, [31272, 38549, 29596, 639, 31390])
(33530, [13275, 9875, 29596, 639, 38678])
(13552, [31390, 38549, 29596, 639, 31272])
(38678, [31272, 13552, 33530, 13275, 29596])
(31257, [14937, 13276, 12615, 30565, 574])

(10803, [4057, 21289, 8913, 17491, 36222])
(39738, [42268, 36330, 23550, 45449, 31364])
(32262, [23326, 4086, 21289, 17491, 8913])
(3194, [8913, 36222, 4057, 25923, 17491])
(42268, [44955, 23551, 22403, 45449, 36330])
(22648, [35606, 4057, 23328, 36222, 17491])
(4086, [23328, 8913, 13146, 26350, 36222])
(22446, [3194, 4086, 36222, 27428, 31414])
(36049, [4086, 33009, 23328, 31414, 4057])
(31414, [23328, 4086, 36222, 31375, 17491])
(27002, [8913, 21289, 23327, 36222, 17491])
(22403, [39738, 46214, 42268, 36330, 23550])
(45156, [45447, 46214, 36330, 44955, 44737])
(30956, [4057, 23328, 4086, 21289, 8913])
(46214, [42268, 45448, 36330, 43788, 44929])
(15896, [4057, 8913, 23328, 3194, 27428])
(38036, [40292, 36330, 45447, 23551, 43788])
(28342, [3194, 4065, 23326, 31414, 4057])
(22445, [4086, 8913, 17491, 21289, 4057])
(26798, [17491, 4086, 23328, 8913, 21289])
(40083, [39738, 36330, 46214, 43787, 44737])
(41147, [45447, 42268, 44955, 36330, 23551])
(31364, [43788, 39738, 36330, 46214, 380

(37860, [42003, 38296, 38968, 36862, 38032])
(37138, [28617, 29329, 32891, 3351, 37611])
(40454, [27991, 39065, 45741, 35888, 29128])
(45426, [43935, 44882, 38424, 41239, 44453])
(40515, [31122, 30621, 33819, 19292, 37981])
(40537, [36360, 36546, 27308, 35722, 41999])
(28652, [30185, 19292, 33819, 23024, 30621])
(36160, [32571, 21645, 32891, 38449, 29329])
(40631, [27308, 36360, 41999, 35722, 36546])
(37850, [31635, 34390, 32740, 32786, 38511])
(20327, [32091, 18818, 31421, 40677, 31358])
(16863, [10277, 4947, 33980, 1416, 8618])
(12848, [23024, 11304, 19292, 7790, 30621])
(37143, [42592, 39023, 38017, 36062, 38296])
(46155, [46147, 43325, 43858, 44882, 43949])
(20980, [12841, 29119, 30621, 11304, 19292])
(39059, [37916, 43978, 44422, 42003, 40250])
(21851, [12844, 7604, 30621, 23024, 19292])
(25911, [21261, 25111, 5876, 1356, 18475])
(43324, [43858, 46148, 45917, 45469, 44882])
(40869, [27308, 36360, 36546, 35722, 41999])
(27655, [38978, 37369, 26786, 38503, 32281])
(46025, [45469, 43

(37432, [38009, 30443, 28848, 37496, 31424])
(28027, [29131, 26408, 26925, 12888, 28027])
(30443, [28848, 31431, 31426, 38009, 37496])
(40069, [43164, 42493, 40601, 45086, 44936])
(37272, [44471, 40066, 42083, 37434, 45381])
(27072, [10587, 34006, 30563, 35612, 28259])
(44296, [44936, 45086, 28870, 40070, 43164])
(44970, [46014, 28871, 34149, 45913, 45897])
(34155, [38513, 45897, 40069, 39984, 44118])
(9104, [10505, 20800, 11211, 9382, 8095])
(37433, [19959, 40601, 26753, 42493, 39785])
(10560, [11898, 10171, 10548, 5542, 6278])
(34008, [28259, 27071, 38139, 29132, 37594])
(34158, [37594, 36820, 34723, 37458, 37185])
(34153, [36801, 27072, 28259, 34167, 37432])
(44471, [42083, 40066, 33745, 37434, 45381])
(42080, [11211, 17425, 36555, 30256, 28496])
(35454, [7777, 9265, 28848, 18361, 10171])
(31431, [30443, 31426, 36800, 38009, 28848])
(8555, [34157, 22944, 30607, 36820, 31])
(38138, [39984, 39203, 26062, 34152, 39785])
(44975, [33745, 40072, 45381, 37434, 45914])
(37271, [35612, 7003,

(24766, [21425, 2559, 15264, 21728, 26768])
(22589, [21424, 4861, 21728, 15220, 31568])
(45368, [34950, 20431, 35893, 36501, 1368])
(4892, [8007, 15220, 35110, 31568, 21424])
(24354, [25753, 32238, 28720, 3827, 24811])
(22486, [24821, 25758, 3833, 36142, 26543])
(46039, [41216, 41661, 31477, 45929, 43946])
(42071, [45229, 42681, 37000, 25100, 37177])
(36128, [29900, 31028, 23929, 29602, 36131])
(22663, [11625, 25365, 17868, 29759, 20695])
(41215, [27973, 29029, 41343, 44834, 34891])
(22598, [8022, 15240, 4873, 24407, 2357])
(42954, [22591, 31542, 4869, 21422, 30100])
(1476, [3358, 21374, 6107, 33053, 3349])
(15160, [2356, 8050, 28710, 24410, 6693])
(5247, [23167, 29461, 19616, 25561, 744])
(19926, [32343, 39174, 29031, 39082, 29667])
(22593, [6672, 15167, 8040, 3288, 10148])
(15174, [6693, 22593, 28710, 24410, 8050])
(42822, [40102, 43302, 44821, 31660, 41454])
(36434, [35652, 35710, 41467, 15606, 43144])
(3288, [4802, 10148, 2555, 24414, 15268])
(10062, [1364, 24301, 33841, 31685, 123

...

(43877, [41387, 45154, 39513, 38504, 29230])
(40470, [24315, 45277, 42589, 39513, 41152])
(40742, [39513, 5141, 38997, 41388, 43145])
(1095, [22838, 23066, 7746, 7167, 2339])
(26829, [29587, 26737, 31324, 26209, 30188])
(4088, [23332, 9468, 21869, 11799, 2339])
(35084, [36583, 32639, 4552, 25011, 6729])
(24332, [20651, 7746, 4392, 11799, 27486])
(23468, [25308, 34353, 35398, 30805, 36241])
(26493, [17863, 33976, 24477, 26514, 22408])
(19225, [8525, 18805, 21276, 9468, 33976])
(31589, [31515, 37522, 34352, 38648, 23470])
(24760, [24760, 28429, 7615, 12687, 10741])
(6329, [2339, 38329, 24477, 23066, 7167])
(40125, [41277, 31522, 45189, 21592, 43743])
(41156, [31525, 41387, 40471, 42589, 24304])
(20255, [2339, 7167, 25004, 23162, 3956])
(6328, [23332, 7167, 24477, 38331, 2339])
(24330, [5469, 16145, 8516, 27486, 24476])
(45187, [42893, 41523, 45800, 35377, 43072])
(36728, [33976, 11662, 18994, 27094, 37671])
(24852, [26804, 1089, 22732, 7167, 24477])
(3558, [22838, 2339, 23066, 2741, 7746

(33152, [37417, 31074, 30640, 33095, 30547])
(40156, [46127, 27251, 44596, 42066, 37481])
(33134, [31074, 30640, 37417, 33095, 30547])
(43255, [43253, 42849, 43709, 42772, 42009])
(14015, [3206, 10956, 19564, 29380, 35291])
(21331, [23234, 34049, 23715, 22644, 26683])
(5102, [22397, 13933, 19562, 7960, 19631])
(20006, [20621, 11522, 25334, 22394, 11679])
(44147, [42066, 43253, 40158, 42772, 39041])
(41981, [46068, 35466, 39481, 44436, 43199])
(13117, [10938, 20757, 33125, 30609, 2886])
(37158, [31634, 38642, 20909, 27923, 34944])
(31203, [8351, 33117, 30609, 3736, 13109])
(17297, [16608, 2402, 16559, 16995, 16656])
(13056, [10938, 20757, 33125, 30609, 2886])
(24473, [19599, 7962, 30153, 19477, 35323])
(38823, [38163, 39483, 34941, 41172, 42487])
(42874, [43279, 43351, 40973, 34120, 43482])
(46127, [37481, 44596, 40156, 45969, 45276])
(37970, [23010, 36937, 20192, 34121, 11681])
(15741, [30000, 29842, 20577, 16913, 6732])
(9311, [7865, 10169, 20694, 2927, 12452])
(35740, [16914, 42488, 

(25796, [27732, 32290, 9194, 17953, 33252])
(33214, [30545, 31074, 30640, 37417, 33095])
(33209, [33095, 37417, 30545, 30640, 31074])
(36521, [25171, 27210, 1285, 18391, 27722])
(43482, [40970, 45398, 44312, 43322, 42874])
(44771, [35849, 33233, 39039, 39484, 40160])
(33237, [44771, 43059, 39484, 45207, 40160])
(13872, [16996, 19625, 7507, 25161, 16573])
(41213, [39487, 42043, 39043, 40632, 37487])
(34123, [33930, 13511, 36937, 37969, 27475])
(4037, [8, 5794, 16983, 22279, 4421])
(45939, [23287, 27240, 46126, 46221, 30090])
(16570, [5702, 16575, 4754, 16566, 24657])
(22263, [23502, 35177, 10954, 17296, 25172])
(11040, [25882, 23063, 20355, 18486, 11290])
(23287, [38476, 34936, 35082, 34125, 30476])
(39415, [39751, 42488, 44148, 41172, 43253])
(16901, [16620, 14048, 22085, 19537, 10111])
(45888, [42764, 37859, 29009, 38951, 41581])
(45200, [40161, 27924, 46115, 42326, 40479])
(44125, [45972, 42908, 45822, 45604, 44596])
(28978, [35603, 26466, 34550, 38902, 37489])
(8968, [6020, 24404, 3

(30171, [8827, 30752, 9741, 31454, 34044])
(45581, [44902, 40206, 42875, 44029, 41914])
(8354, [17476, 5453, 5449, 4457, 10593])
(39093, [41688, 34529, 38264, 36202, 39313])
(43441, [43441, 43439, 27191, 43440, 40098])
(42180, [27179, 31371, 41038, 40208, 42519])
(46377, [38264, 37536, 45766, 45752, 45934])
(37500, [40483, 15142, 6416, 8827, 37607])
(38264, [10965, 37536, 42746, 41994, 40240])
(12286, [5293, 14909, 5401, 21664, 17477])
(36205, [39093, 39313, 34527, 41688, 21323])
(41037, [39786, 34910, 35412, 44517, 32365])
(21054, [38222, 45351, 42518, 35808, 41770])
(17476, [9745, 15689, 10581, 7466, 4457])
(42179, [42521, 45748, 46385, 31371, 40208])
(31307, [21664, 5293, 14909, 5452, 17477])
(20403, [8827, 37607, 6416, 30174, 36231])
(27171, [43449, 25576, 39093, 10965, 37739])
(43449, [39786, 28036, 36202, 38264, 34527])
(36198, [39093, 41688, 39950, 27172, 40851])
(5349, [5449, 32904, 17476, 8666, 10593])
(31355, [46114, 45823, 44728, 45766, 46384])
(10570, [10608, 5502, 5631, 17

(7616, [21664, 5293, 10572, 14909, 5401])
(10582, [28638, 5631, 7420, 17476, 15689])
(29713, [38264, 41688, 38011, 40240, 34910])
(36201, [25765, 27171, 28061, 39093, 25576])
(27185, [39093, 28036, 31351, 28063, 25576])
(5161, [10577, 5157, 10608, 8354, 5450])
(25767, [27318, 36439, 6416, 18184, 28680])
(37828, [42518, 40205, 44029, 44902, 41914])
(9744, [5632, 17476, 7468, 5401, 15689])
(27191, [43436, 43438, 40098, 27191, 43435])
(35808, [21054, 38222, 41770, 42518, 45350])
(7467, [4459, 5293, 11051, 28638, 14909])
(40206, [45584, 41914, 45350, 42518, 44029])
(6488, [14909, 5401, 20402, 7616, 9745])
(40240, [41994, 41688, 36202, 42879, 38264])
(42182, [41038, 27179, 40208, 42519, 31371])
(44517, [39949, 32365, 45135, 34910, 39786])
(27180, [40996, 35810, 31371, 38223, 41038])
(27182, [28063, 25576, 28036, 39093, 10965])
(41914, [45584, 45350, 41099, 42518, 44029])
(34529, [25576, 43449, 37739, 36202, 39093])
(39949, [38011, 41688, 46377, 43304, 39787])
(5195, [10577, 5157, 5450, 5728

In [14]:
with conn.cursor() as cursor:
    conn.autocommit = True
    dropcreate = sql.SQL('''DROP TABLE IF EXISTS test.portal_obj_recs; 
    CREATE TABLE test.portal_obj_recs (obj_id BIGINT NOT NULL,
    recs_id BIGINT [] NOT NULL, CONSTRAINT portal_obj_recs_pkey PRIMARY KEY (obj_id))''')
    cursor.execute(dropcreate)
    insert = sql.SQL('INSERT INTO test.portal_obj_recs (obj_id, recs_id) VALUES {}').format(
        sql.SQL(',').join(map(sql.Literal, values))
    )
    cursor.execute(insert)